KAGGLE ile hibrit model şeklinde gidilmektedir. Veri işlemesi lokalde yapılırken, Model eğitimi kaggle üzerinden yapılmaktadır.

Bu dosya kaggle da eğitilen modelin kodlarını ve gelişimini arşivlemek açısından birebir kod ve açıklamalarıdır. (Çıktıları Kendiniz Çalıştırabilirsiniz)

In [ ]:
""" VERİ YÜKLEME """

import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt

# model için hazırladığımız train.data verisi seti
df = pd.read_csv("/kaggle/input/instacart-features-local/train_data.csv")

print(f"✅ Veri yüklendi! Boyut: {df.shape}")
df.head()

Create_Dataset dosyası ile son hale getirdiğimiz eğitim için veri setini sisteme yükledik ve burada çekiyoruz.

In [ ]:
""" DEĞİŞKENLERİN HAZIRLANMASI """

X = df.drop(['user_id', 'product_id', 'reordered'], axis=1) # değişkenler
y = df['reordered'] # hedef

# %80 Eğitim, %20 test
print("🔪 Veri setleri ayrılıyor...")
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Eğitim Seti: {X_train.shape}")
print(f"Test Seti:   {X_val.shape}")

Modelin kullanıcı, ürün ve tahmin etmesi gereken (hedef değişken) sütunlarını görmemesi için kaldırıyoruz diğerlerini alacak. eğer kaldırmaz isek model bunları baz alarak öğrenmek yerine ezber yapar. Yani ahmet şunu almış, muz bu kadar alınıyor gibi ezberden kaçınsın. Eğer ürün popülerse ve kullanıcı daha önce çok aldıysa cevap Evet' yani tekrar alır demesi lazım bu mantıkla yaklaşmalı.


In [ ]:
""" XGBOOST MODELİNİN AYARLARI """

model = xgb.XGBClassifier(
    objective='binary:logistic', # İkili sınıflandırmadır. (binary ikili durumu ifade eder evet-hayır,alacak-almıycak gibi, logistic ise sınıflandırma için kullanırız direkt 0,1 vermesin küsüratlıda versin)
    eval_metric='logloss', # puan sistemi (Çok eminse ve yanlış çıkarsa büyük ceza gibi tahminle orantılı ceza)
    n_estimators=1000,      # Ağaç sayısı
    learning_rate=0.05,     # Öğrenme hızı
    max_depth=6,            # Ağaç derinliği
    tree_method='hist',     # Hızlandırma için (histogram ile veriyi gruplayarak işlem yapsın. hız katar.)
    device='cuda',          # GPU kullanımı (Eğer GPU açtıysan 'cuda', açmadıysan 'cpu' yap)
    early_stopping_rounds=50 # Eğer 50 tur boyunca iyileşme olmazsa dur
)

print("🚀 Model eğitimi başlıyor...")
model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)], # her ağaçtan sonra görmediği verilerle test yapra
    verbose=100  # Her 100 adımda(ağaçta) bir durum raporu ver
)

In [ ]:
""" TAHMİN """

# Tahmin yap
y_pred = model.predict(X_val) # x_val verileri ile tahmin ettirme
# 0 ve 1 leden oluşan tahmin sonuçları y_pred de

# Raporla
print("📊 Model Performansı:")
print(classification_report(y_val, y_pred)) # y_val gerçek cevaplar ile y_pred tahminlerin karşılaştırılması

# Hangi özellik en önemlisi (Interpretabilitiy)
xgb.plot_importance(model, max_num_features=10)  # karar verirken en çok kullandığı özellikleri sıralar
plt.title("Hangi Özellik Daha Önemli?")
plt.show()

predict() metodunda varsayılan olarak eşik değer %50 dir. Yani 1 (alır) sonucunu vermesi için %50.01 ikna olması yeterlidir.

classification_report() metodu ceapların kontrolünü yapar. Direkt şu kadar bildi (accuracy) ölçmez genel bir test yapar. 

Testte Precision (Kesinlik): "Alacak" dediklerinin yüzde kaçı gerçekten aldı

Recall (Duyarlılık - En Önemlisi): Gerçekten alanların yüzde kaçını yakalayabildi

F1-Score: Precision ve Recall'un ortalaması. Modelin genel dengesi. (direkt bu metrik çok şey ifade eder)

Support: o sınıftan (0 veya 1) test setinde kaç tane örnek olduğu.

Interpretability kısmı analiz için çok önemlidir. İlk sonuçlar genelde hiç iç açıcı olmayakcaktır. Bunun tedavisinide en kısa ve kesin yoldan bu adım ile saptayabilir. Modele bu kararları verirken hangi özellikleri kullandın diye sorar. Bu da model gerçekten bizim istediğimiz mantık da düşünüp düşünmediğini gösterir ve nokta atışı tedavi yapabiliriz.

In [ ]:
import numpy as np
from sklearn.metrics import f1_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# 1-Tahminlerin ihtimal oranları (yani karar verirken ne kadar oranlar ile verdin)
print("Olasılıklar hesaplanıyor")
y_pred_probs = model.predict_proba(X_val)[:, 1]


# 2-En iyi eşik değerini (Threshold) bulma döngüsü
best_threshold = 0
best_f1 = 0
results = []

print("En iyi eşik değeri aranıyor")

# 0.10'dan 0.50'ye kadar 0.02 adım adım dene
for threshold in np.arange(0.1, 0.51, 0.02):
    # Olasılık > threshold ise 1 yap, değilse 0
    preds_custom = (y_pred_probs > threshold).astype(int) # karşılaştırma
    
    # F1 Skorunu hesapla (Class 1 için)
    f1 = f1_score(y_val, preds_custom) # her eşiğe göre hesaplamadığımız sınıf ile gerçek değerler f1 skorunu hesapla
    results.append((threshold, f1)) # bu ölçümleri karşılaştırmak üzere listeye atıyoruz
    
    print(f"   Eşik: {threshold:.2f} -> F1 Score: {f1:.4f}") # her döngü sonucu
    
    # en iyi f1 skoru seçimi
    if f1 > best_f1:
        best_f1 = f1
        best_threshold = threshold

print("-" * 30)
print(f"EN İYİ EŞİK DEĞERİ: {best_threshold:.2f}")
print(f"Yeni F1 Skoru: {best_f1:.4f}")


# 3-En iyi eşik ile DETAYLI raporu bas
final_preds = (y_pred_probs > best_threshold).astype(int) # en iyi eşik değeri al

print("\nOptimize Edilmiş Rapor:")
print(classification_report(y_val, final_preds)) # alınan değerler tekrardan report metodu ile raporla


# 4-Karışıklık Matrisi (Confusion Matrix) - Neyi ne kadar doğru bildik görmek için
cm = confusion_matrix(y_val, final_preds) # cevaplar ile bizim en iyi f1 li tahmnilerimiz
plt.figure(figsize=(6, 4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title(f'Confusion Matrix (Threshold: {best_threshold:.2f})')
plt.ylabel('Gerçek Değer')
plt.xlabel('Tahmin')
plt.show()

predict_proba() ile bu sefer ihtimalleri alıyoruz. yani sen sınıflandırma (0 alma, 1 al) yaparken %50 üstü olunca 1 diyosun ama  ne kadar oran verdin her veri için diye soruyoruz. geriye her tahmini için [[0.3, 0.7], [0.8, 0.2], ...] gibi bir çıktı döner biz de sadece sağdaki yani 1.indeksteki 1(alır) için olan oranları alıyoruz.

pred default olarak threshold değerini %50 alır yani 50.1 olduğu an 1 verir. Burada çok korkak davrandığı tespit ettik çıkan sonuçlarda. Bu değerin en optimum sonuçları ne kadarken verdiğini görmek için tüm değeleri ufak adımlarla değiştire değiştire deniyoruz. ihtimalleri aldığımız kısım ile sürekli denediğimiz eşik değeri karşılaştırıp deniyoruz. En iyi sonucu veren f1 skorlu değerli report fonkisyonu ile detaylı raporluyoruz

Son olarak karışıklık matrisi ile röntgen çekiyoruz. Bu matriste satırlar Gerçek Durumu, sütunlar ise Modelin Tahminini gösterir. Her kutunun ise ayrı bir anlamı vardır. 

Sol üst kutu (TN-TRUE NEGATİVE) Buraya sessiz doğru deneriz. burada müşterinin almayacağı ürüne modelinde almayacak demesidir. saten bu durum her problemde genelde doğrudur. Çünkü bu durumda bulunan bi ton veri saten hep vardır.

Sağ üst kutu (FP-FALSE POSITIVE) Yanlış alarm verdiğimiz durumlar. Müşterinin almayacağı ürüne modelin alacak demesidir. Bu oran son yaptığımız eşik değeri 50 den 20 ye düşürme işlemimizden sonra ciddi arttı. Çünkü model çok daha sık alacak dediği için hata oranıda arttı. Bu sorun bu problem için göz ardı edilebilir çünkü müşteriye ürün önerme maliyeti düşüktür ama bu önermeye karşılık vermesi önemli olduğundan maliyet-kazanç oranımız yüksektir bundan dolayı burada ki hata payı göz ardı edilebilir. (biz az ihtimalde olsa önerelim maliyeti yok ya alırsa gibi)

Sol alt kutu (FN-FALSE NEGATIVE) Bu kısım farkı yaratacağımız, problemin ana amaç kısmıdır. Müşterinin aldığı ancak biz almayacak diyip de önermede bulunmadığımız kısımdır. Eşik değeri azaltmamız burada da ciddi fayda sağladı ancak halen sonuçlara göre yüksek.

Sağ alt kutu (TP-TRUE POSİTİVE) Burası doğru atış yaptığımız yerler. Müşterinin aldığı modelinde alacak dediği kısımdır.